In [1]:
import numpy as np
import pandas as pd

In [2]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
min_rows_categories = None # minimo de objetos que devem conter essas categorias para ser desconsiderado
df_input_path = '../../dados/X_trainToronto.csv'

In [3]:
# Parameters
min_rows_categories = 0
df_input_path = "../../dados/X_trainToronto.csv"
product = {
    "data": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/processar_categories_data.parquet",
    "nb": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/output_processar_categories.ipynb",
}


In [4]:
df_reg = pd.read_csv(df_input_path)
df_reg = df_reg[['business_id', 'categories']]
df_reg

,business_id,categories
0,vHzWmPWHN4J1hRR3W3AMQg,"Wine Bars, Tapas/Small Plates, Restaurants, Ba..."
1,15to24Q-otAHmto7FzsWRg,"Beauty & Spas, Hair Salons, Barbers, Shopping,..."
2,8aqKdf4G4AAir8k_Kdslvg,"Health & Medical, Medical Centers"
3,uxU1vr5AhhkTQ83X0bpeyg,"Health & Medical, Hospitals, Medical Centers"
4,f702hTJoqdR34Jn23C7d1A,"Automotive, Bikes, Shopping, Auto Repair, Spor..."
...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,"Restaurants, Halal, Mexican"
17578,o2811xZ0ufRi7WcsqWiGzw,"Food, Coffee & Tea"
17579,v9T9m_TbpLuaC2Kud5UbLA,"Sewing & Alterations, Shopping, Local Services"
17580,bzGnAIe9OiBHGWJE-dZ_8g,"Chicken Wings, Bars, Nightlife, Restaurants, P..."


## Transformar categories em dummies

In [5]:
df_reg.categories = df_reg.categories.fillna('')
df_reg.categories = df_reg.categories.astype(str)
df_reg.categories = df_reg.categories.str.split(',')
df_reg.categories = df_reg.categories.apply(lambda x: [s.strip().lower() for s in x])
df_reg.categories

0        [wine bars, tapas/small plates, restaurants, b...
1        [beauty & spas, hair salons, barbers, shopping...
2                      [health & medical, medical centers]
3           [health & medical, hospitals, medical centers]
4        [automotive, bikes, shopping, auto repair, spo...
                               ...                        
17577                        [restaurants, halal, mexican]
17578                                 [food, coffee & tea]
17579     [sewing & alterations, shopping, local services]
17580    [chicken wings, bars, nightlife, restaurants, ...
17581    [orthotics, massage therapy, chiropractors, he...
Name: categories, Length: 17582, dtype: object

In [6]:
from collections import defaultdict

all_categories = defaultdict(int)
for c_list in df_reg.categories:
    if type(c_list) is float:
        c_list = []
    for c in c_list:
        all_categories[c.strip().lower()] = all_categories[c.strip().lower()] + 1

best_categories = pd.Series(all_categories)
best_categories = best_categories[best_categories > min_rows_categories]
others_categories = set(all_categories.keys()).difference(set(best_categories.index))
best_categories

wine bars              127
tapas/small plates      94
restaurants           7418
bars                  1443
nightlife             1616
                      ... 
clock repair             1
metal fabricators        1
cabaret                  2
psychologists            1
tui na                   1
Length: 897, dtype: int64

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df_cat_dummies = pd.DataFrame(mlb.fit_transform(df_reg.categories),columns=mlb.classes_, index=df_reg.index)

if min_rows_categories > 0:
    df_cat_dummies['category_others_categories'] = df_cat_dummies.apply(lambda x: x[list(others_categories)].sum(), axis=1)
    df_cat_dummies = df_cat_dummies.drop(list(others_categories), axis=1)


df_reg = pd.concat([df_reg, df_cat_dummies], axis=1)
df_reg = df_reg.rename(columns={cat:'category_' + cat for cat in best_categories.index})
df_reg

,business_id,categories,category_,category_& probates,category_3d printing,category_acai bowls,category_accessories,category_accountants,category_acne treatment,category_active life,...,category_wine & spirits,category_wine bars,category_wine tasting classes,category_wine tours,category_wineries,category_women's clothing,category_wraps,category_yelp events,category_yoga,category_zoos
0,vHzWmPWHN4J1hRR3W3AMQg,"[wine bars, tapas/small plates, restaurants, b...",0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,15to24Q-otAHmto7FzsWRg,"[beauty & spas, hair salons, barbers, shopping...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8aqKdf4G4AAir8k_Kdslvg,"[health & medical, medical centers]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,uxU1vr5AhhkTQ83X0bpeyg,"[health & medical, hospitals, medical centers]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,f702hTJoqdR34Jn23C7d1A,"[automotive, bikes, shopping, auto repair, spo...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,"[restaurants, halal, mexican]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17578,o2811xZ0ufRi7WcsqWiGzw,"[food, coffee & tea]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17579,v9T9m_TbpLuaC2Kud5UbLA,"[sewing & alterations, shopping, local services]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17580,bzGnAIe9OiBHGWJE-dZ_8g,"[chicken wings, bars, nightlife, restaurants, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
category_columns = [x for x in df_reg.columns if x.startswith('category_')]
df_reg[category_columns] = df_reg[category_columns].apply(lambda x: x.astype(float))

In [9]:
df_reg.to_parquet(product['data'])

## Teste manual

In [10]:
from IPython.display import display
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_reg.iloc[13]['categories'])
    display(df_reg.iloc[[13]][category_columns])
    display('----------------------------------')
    display(df_reg.iloc[542]['categories'])
    display(df_reg.iloc[[542]][category_columns])
    display('----------------------------------')
    display(df_reg.iloc[5389]['categories'])
    display(df_reg.iloc[[5389]][category_columns])

['sandwiches', 'american (traditional)', 'breakfast & brunch', 'restaurants']

,category_,category_& probates,category_3d printing,category_acai bowls,category_accessories,category_accountants,category_acne treatment,category_active life,category_acupuncture,category_adult,category_adult education,category_adult entertainment,category_advertising,category_aerial fitness,category_aestheticians,category_afghan,category_african,category_airlines,category_airport lounges,category_airport shuttles,category_airports,category_airsoft,category_alternative medicine,category_amateur sports teams,category_american (new),category_american (traditional),category_amusement parks,category_animal shelters,category_antiques,category_apartments,category_appliances,category_appliances & repair,category_appraisal services,category_aquarium services,category_aquariums,category_arabian,category_arcades,category_archery,category_architects,category_argentine,category_art classes,category_art galleries,category_art museums,category_art schools,category_art supplies,category_arts & crafts,category_arts & entertainment,category_asian fusion,category_astrologers,category_auction houses,category_audiologist,category_australian,category_austrian,category_auto customization,category_auto detailing,category_auto glass services,category_auto loan providers,category_auto parts & supplies,category_auto repair,category_automotive,category_awnings,category_axe throwing,category_ayurveda,category_baby gear & furniture,category_badminton,category_bagels,category_bakeries,category_balloon services,category_bangladeshi,category_bankruptcy law,category_banks & credit unions,category_barbeque,category_barbers,category_barre classes,category_bars,category_bartenders,category_bartending schools,category_baseball fields,category_basque,category_beach equipment rentals,category_beaches,category_beauty & spas,category_bed & breakfast,category_beer,category_beer bar,category_beer gardens,category_beer hall,category_beer tours,category_belgian,category_bespoke clothing,category_bicycles,category_bike rentals,category_bike repair/maintenance,category_bike sharing,category_bike shop,category_bike tours,category_bikes,category_bistros,category_blow dry/out services,category_boat charters,category_boat repair,category_boat tours,category_boating,category_bocce ball,category_body contouring,category_body shops,category_bookbinding,category_books,category_bookstores,category_boot camps,category_botanical gardens,category_boudoir photography,category_bowling,category_boxing,category_brasseries,category_brazilian,category_brazilian jiu-jitsu,category_breakfast & brunch,category_breweries,category_brewing supplies,category_brewpubs,category_bridal,category_british,category_bubble soccer,category_bubble tea,category_buddhist temples,category_buffets,category_building supplies,category_burgers,category_burmese,category_bus rental,category_bus tours,category_buses,category_business consulting,category_business law,category_butcher,category_cabaret,category_cabinetry,category_cafes,category_cafeteria,category_cajun/creole,category_cambodian,category_campgrounds,category_canadian (new),category_candy stores,category_cannabis clinics,category_cantonese,category_car dealers,category_car rental,category_car share services,category_car stereo installation,category_car wash,category_car window tinting,category_cardio classes,category_cardiologists,category_cards & stationery,category_career counseling,category_caribbean,category_carpenters,category_carpet cleaning,category_carpet installation,category_carpeting,category_casinos,category_caterers,category_challenge courses,category_champagne bars,category_check cashing/pay-day loans,category_cheese shops,category_cheesesteaks,category_chicken shop,category_chicken wings,category_child care & day care,category_children's clothing,category_chinese,category_chiropractors,category_chocolatiers & shops,category_churches,category_cideries,category_cinema,category_civic center,category_climbing,category_clock repair,category_clo

'----------------------------------'

['food', 'cheese shops', 'specialty food']

,category_,category_& probates,category_3d printing,category_acai bowls,category_accessories,category_accountants,category_acne treatment,category_active life,category_acupuncture,category_adult,category_adult education,category_adult entertainment,category_advertising,category_aerial fitness,category_aestheticians,category_afghan,category_african,category_airlines,category_airport lounges,category_airport shuttles,category_airports,category_airsoft,category_alternative medicine,category_amateur sports teams,category_american (new),category_american (traditional),category_amusement parks,category_animal shelters,category_antiques,category_apartments,category_appliances,category_appliances & repair,category_appraisal services,category_aquarium services,category_aquariums,category_arabian,category_arcades,category_archery,category_architects,category_argentine,category_art classes,category_art galleries,category_art museums,category_art schools,category_art supplies,category_arts & crafts,category_arts & entertainment,category_asian fusion,category_astrologers,category_auction houses,category_audiologist,category_australian,category_austrian,category_auto customization,category_auto detailing,category_auto glass services,category_auto loan providers,category_auto parts & supplies,category_auto repair,category_automotive,category_awnings,category_axe throwing,category_ayurveda,category_baby gear & furniture,category_badminton,category_bagels,category_bakeries,category_balloon services,category_bangladeshi,category_bankruptcy law,category_banks & credit unions,category_barbeque,category_barbers,category_barre classes,category_bars,category_bartenders,category_bartending schools,category_baseball fields,category_basque,category_beach equipment rentals,category_beaches,category_beauty & spas,category_bed & breakfast,category_beer,category_beer bar,category_beer gardens,category_beer hall,category_beer tours,category_belgian,category_bespoke clothing,category_bicycles,category_bike rentals,category_bike repair/maintenance,category_bike sharing,category_bike shop,category_bike tours,category_bikes,category_bistros,category_blow dry/out services,category_boat charters,category_boat repair,category_boat tours,category_boating,category_bocce ball,category_body contouring,category_body shops,category_bookbinding,category_books,category_bookstores,category_boot camps,category_botanical gardens,category_boudoir photography,category_bowling,category_boxing,category_brasseries,category_brazilian,category_brazilian jiu-jitsu,category_breakfast & brunch,category_breweries,category_brewing supplies,category_brewpubs,category_bridal,category_british,category_bubble soccer,category_bubble tea,category_buddhist temples,category_buffets,category_building supplies,category_burgers,category_burmese,category_bus rental,category_bus tours,category_buses,category_business consulting,category_business law,category_butcher,category_cabaret,category_cabinetry,category_cafes,category_cafeteria,category_cajun/creole,category_cambodian,category_campgrounds,category_canadian (new),category_candy stores,category_cannabis clinics,category_cantonese,category_car dealers,category_car rental,category_car share services,category_car stereo installation,category_car wash,category_car window tinting,category_cardio classes,category_cardiologists,category_cards & stationery,category_career counseling,category_caribbean,category_carpenters,category_carpet cleaning,category_carpet installation,category_carpeting,category_casinos,category_caterers,category_challenge courses,category_champagne bars,category_check cashing/pay-day loans,category_cheese shops,category_cheesesteaks,category_chicken shop,category_chicken wings,category_child care & day care,category_children's clothing,category_chinese,category_chiropractors,category_chocolatiers & shops,category_churches,category_cideries,category_cinema,category_civic center,category_climbing,category_clock repair,category_clo

'----------------------------------'

['organic stores', 'health markets', 'specialty food', 'grocery', 'food']

,category_,category_& probates,category_3d printing,category_acai bowls,category_accessories,category_accountants,category_acne treatment,category_active life,category_acupuncture,category_adult,category_adult education,category_adult entertainment,category_advertising,category_aerial fitness,category_aestheticians,category_afghan,category_african,category_airlines,category_airport lounges,category_airport shuttles,category_airports,category_airsoft,category_alternative medicine,category_amateur sports teams,category_american (new),category_american (traditional),category_amusement parks,category_animal shelters,category_antiques,category_apartments,category_appliances,category_appliances & repair,category_appraisal services,category_aquarium services,category_aquariums,category_arabian,category_arcades,category_archery,category_architects,category_argentine,category_art classes,category_art galleries,category_art museums,category_art schools,category_art supplies,category_arts & crafts,category_arts & entertainment,category_asian fusion,category_astrologers,category_auction houses,category_audiologist,category_australian,category_austrian,category_auto customization,category_auto detailing,category_auto glass services,category_auto loan providers,category_auto parts & supplies,category_auto repair,category_automotive,category_awnings,category_axe throwing,category_ayurveda,category_baby gear & furniture,category_badminton,category_bagels,category_bakeries,category_balloon services,category_bangladeshi,category_bankruptcy law,category_banks & credit unions,category_barbeque,category_barbers,category_barre classes,category_bars,category_bartenders,category_bartending schools,category_baseball fields,category_basque,category_beach equipment rentals,category_beaches,category_beauty & spas,category_bed & breakfast,category_beer,category_beer bar,category_beer gardens,category_beer hall,category_beer tours,category_belgian,category_bespoke clothing,category_bicycles,category_bike rentals,category_bike repair/maintenance,category_bike sharing,category_bike shop,category_bike tours,category_bikes,category_bistros,category_blow dry/out services,category_boat charters,category_boat repair,category_boat tours,category_boating,category_bocce ball,category_body contouring,category_body shops,category_bookbinding,category_books,category_bookstores,category_boot camps,category_botanical gardens,category_boudoir photography,category_bowling,category_boxing,category_brasseries,category_brazilian,category_brazilian jiu-jitsu,category_breakfast & brunch,category_breweries,category_brewing supplies,category_brewpubs,category_bridal,category_british,category_bubble soccer,category_bubble tea,category_buddhist temples,category_buffets,category_building supplies,category_burgers,category_burmese,category_bus rental,category_bus tours,category_buses,category_business consulting,category_business law,category_butcher,category_cabaret,category_cabinetry,category_cafes,category_cafeteria,category_cajun/creole,category_cambodian,category_campgrounds,category_canadian (new),category_candy stores,category_cannabis clinics,category_cantonese,category_car dealers,category_car rental,category_car share services,category_car stereo installation,category_car wash,category_car window tinting,category_cardio classes,category_cardiologists,category_cards & stationery,category_career counseling,category_caribbean,category_carpenters,category_carpet cleaning,category_carpet installation,category_carpeting,category_casinos,category_caterers,category_challenge courses,category_champagne bars,category_check cashing/pay-day loans,category_cheese shops,category_cheesesteaks,category_chicken shop,category_chicken wings,category_child care & day care,category_children's clothing,category_chinese,category_chiropractors,category_chocolatiers & shops,category_churches,category_cideries,category_cinema,category_civic center,category_climbing,category_clock repair,category_clo